## Lets "pivot" data and nest it into a list.
We currently have our tracks_by_album table modeled as such:

````
CREATE TABLE music.tracks_by_album (
    album_title text,
    album_year int,
    track_number int,
    album_genre text static,
    performer text static,
    track_title text,
    PRIMARY KEY ((album_title, album_year), track_number)
````

Let's model it as one partition per album, and the track titles in a list


In [31]:
%%Cql create table if not exists music.albums (album_title text,
album_genre text,
album_year int,
performer text,
tracks list<text>, 
primary key ((album_title, album_year)) )

In [59]:
%%Cql truncate music.albums

### Now a handy case class

In [47]:
case class Track(album_title: String,
album_year:Int, number:Int,
album_genre: String,
performer: String,
track_title: String)

### And an RDD to pick up the original data

In [48]:
val tracks = sc.cassandraTable("music","tracks_by_album").as(Track)

In [49]:
tracks.first

Track(Duos For Violin and Cello,2000,1,Classical,Nigel Kennedy,Sonata for Violin and Cello - Allegro)

## Create a Pair RDD with key, track name.  The key in this case (album_title, year).  So we want tuples ((album_title, year), track_title).  Also filter out tracks that are null

In [51]:
val track_pairs = tracks.filter(t => t.track_title != null).map(t => ((t.album_title, t.album_year), t.track_title))

In [36]:
track_pairs.first.toString

## Use groupByBey, to convert it to (key, collection of track names)

In [52]:
val albums_with_tracks = track_pairs.groupByKey

In [38]:
albums_with_tracks.first.toString


## Flatten it out so we have (album_title, year, collection of tracks)

In [53]:
val albums_with_tracks2 = albums_with_tracks.map{ case ((album, year), tracks) => (album, year, tracks) }

In [54]:
albums_with_tracks2.first.toString

(Bramble Rose,2002,CompactBuffer(Trouble Over Me, Virginia, No One Can Warn You, Neighborhood, Bird Of Freedom, Bramble Rose, I Know Him Too, Sunday, Supposed To Make You Happy, Diamond Shoes, Are You Still In Love With Me?, When I Cross Over))

In [61]:
albums_with_tracks2.saveToCassandra("music","albums", SomeColumns("album_title", "album_year", "tracks"))

## ... And check it out

In [62]:
%%Cql select * from music.albums limit 5;

album_title,album_year,album_genre,performer,tracks
Duos For Violin and Cello,2000,<null>,<null>,"[Sonata for Violin and Cello - Allegro, Sonata for Violin and Cello - Tres vif, Sonata for Violin and Cello - Lent, Sonata for Violin and Cello - Vif, avec entrain, Passacaglia, Duo for Violin and Cello Op. 7 - Allegro serioso, non troppo, Duo for Violin and Cello Op. 7 - Adagio-Andante-Tempo I, Duo for Violin and Cello Op. 7 - Maestoso e largamente, ma non troppo lento-Presto, Two-Part Intervention No. 6 in E]"
Golden Boy Elvis,1981,<null>,<null>,"[She's Not You, Return To Sender, Joshua Fit The Battle, Don't, Tutti Frutti, It's Now Or Never, Surrender, Do The Clam, Kiss Me Quick, Such A Night, Blueberry Hill, Don't Be Cruel, Heartbreak Hotel, Fun In Acapulco, Hound Dog, Wooden Heart]"
Home,1995,<null>,<null>,"[I Believe, Let Me Be The One, All Along, Oh Virginia, Nora, Would You Be There, Home, End Of The World, Heaven, Forever For Tonight, Lucky To Be Here]"
Little Sweet Delirium,1996,<null>,<null>,"[Chase, Revolution, 829, Reaction, Hands Like Wings, God Shaped Hole, Fall, Point of You]"
"Merry Christmas From Elvis Presley, A",1982,<null>,<null>,"[O Come All Ye Faithful, The First Noel, On A Snowy Christmas Night, Winter Wonderland, The Wonderful World Of Christmas, It Won't Seem Like Christmas, I'll Be Home On Christmas Day, If I Get Home On Christmas Day, Holly Leaves And Christmas Trees, Merry Christmas Baby, Silver Bells]"


## Now we need to fill in genre and year.  Let's just upsert them in!  This will also insert any albums with no tracks

In [63]:
tracks.saveToCassandra("music","albums",SomeColumns("album_title","album_year","album_genre","performer"))

In [64]:
%%Cql select * from music.albums limit 5;

album_title,album_year,album_genre,performer,tracks
Duos For Violin and Cello,2000,Classical,Nigel Kennedy,"[Sonata for Violin and Cello - Allegro, Sonata for Violin and Cello - Tres vif, Sonata for Violin and Cello - Lent, Sonata for Violin and Cello - Vif, avec entrain, Passacaglia, Duo for Violin and Cello Op. 7 - Allegro serioso, non troppo, Duo for Violin and Cello Op. 7 - Adagio-Andante-Tempo I, Duo for Violin and Cello Op. 7 - Maestoso e largamente, ma non troppo lento-Presto, Two-Part Intervention No. 6 in E]"
Golden Boy Elvis,1981,Rock,Elvis Presley,"[She's Not You, Return To Sender, Joshua Fit The Battle, Don't, Tutti Frutti, It's Now Or Never, Surrender, Do The Clam, Kiss Me Quick, Such A Night, Blueberry Hill, Don't Be Cruel, Heartbreak Hotel, Fun In Acapulco, Hound Dog, Wooden Heart]"
Home,1995,Rock,Blessid Union of Souls,"[I Believe, Let Me Be The One, All Along, Oh Virginia, Nora, Would You Be There, Home, End Of The World, Heaven, Forever For Tonight, Lucky To Be Here]"
Little Sweet Delirium,1996,Unknown,Creeker,"[Chase, Revolution, 829, Reaction, Hands Like Wings, God Shaped Hole, Fall, Point of You]"
"Merry Christmas From Elvis Presley, A",1982,Rock,Elvis Presley,"[O Come All Ye Faithful, The First Noel, On A Snowy Christmas Night, Winter Wonderland, The Wonderful World Of Christmas, It Won't Seem Like Christmas, I'll Be Home On Christmas Day, If I Get Home On Christmas Day, Holly Leaves And Christmas Trees, Merry Christmas Baby, Silver Bells]"
